# YouTube Ranking

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [2]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube'
driver = webdriver.Chrome('chromedriver')
driver.get(url)
time.sleep(2)

* step1, 2

In [4]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('tr.aos-init')
len(trs)

100

* step3

In [5]:
#랭크정보
tr = trs[0]
tr.select_one('.rank').get_text().strip()

'1'

In [17]:
#카테고리 정보
tr.select_one('.category').get_text().strip()[1:-1]

'게임'

In [7]:
#채널명
tr.select_one('.subject a').get_text().strip()

'BLACKPINK'

In [8]:
#구독자, 조회수, 비디오갯수
subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
view = tr.select_one('.view_cnt').get_text().strip()
video = tr.select_one('.video_cnt').get_text().strip()
subscriber, view, video

('8380만', '286억1398만', '466개')

In [31]:
lines = []
for tr in trs:
    rank = tr.select_one('.rank').get_text().strip()
    category = tr.select_one('.category').get_text().strip()[1:-1]
    chennel = tr.select_one('.subject a').get_text().strip()
    subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
    view = tr.select_one('.view_cnt').get_text().strip()
    video = tr.select_one('.video_cnt').get_text().strip()
    lines.append([rank, category, chennel, subscriber, view, video])


In [32]:
df = pd.DataFrame(lines, columns = ['순위', '카테고리', '채널', '구독자수', '조회수', '비디오수'])

In [33]:
df

,순위,카테고리,채널,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,8380만,286억1398만,466개
1,2,음악/댄스/가수,BANGTANTV,7300만,191억4327만,"2,086개"
2,3,음악/댄스/가수,HYBE LABELS,6950만,258억4674만,"1,065개"
3,4,음악/댄스/가수,SMTOWN,3130만,262억1822만,"4,052개"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223개
...,...,...,...,...,...,...
95,96,TV/방송,JTBC Drama,469만,50억8639만,"27,000개"
96,98,키즈/어린이,CoCosToy 꼬꼬스토이,467만,28억7649만,656개
97,97,음식/요리/레시피,Cooking tree 쿠킹트리,467만,4억6943만,"1,410개"
98,99,음식/요리/레시피,까니짱 [ G-NI : 蟹ちゃん],465만,13억5018만,719개


In [37]:
# 286억1398만 -> 28,613,980,000
def convert_unit(s):
    s = s.replace('억', '').replace('개', '').replace(',', '').replace('만', '0000')
    return f'{int(s):,d}'

In [38]:
convert_unit('286억1398만')

'28,613,980,000'

In [28]:
# df['구독자수'] = df['구독자수'].apply(convert_unit)
# # , 조회수, 비디오수

In [39]:
for column in df.columns[3:]:
    df[column] = df[column].apply(convert_unit)
df.head()

,순위,카테고리,채널,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,"83,800,000","28,613,980,000",466
1,2,음악/댄스/가수,BANGTANTV,"73,000,000","19,143,270,000","2,086"
2,3,음악/댄스/가수,HYBE LABELS,"69,500,000","25,846,740,000","1,065"
3,4,음악/댄스/가수,SMTOWN,"31,300,000","26,218,220,000","4,052"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223


* 모든페이지

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
lines = []
for page in tqdm(range(1,11)):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('tr.aos-init')
    
    for tr in trs:
        rank = int(tr.select_one('.rank').get_text().strip())
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one('.subject a').get_text().strip()
        subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
        view = tr.select_one('.view_cnt').get_text().strip()
        video = tr.select_one('.video_cnt').get_text().strip()
        lines.append([rank, category, channel, subscriber, view, video])

In [ ]:
df = pd.DataFrame(lines, columns=['순위','카테고리','채널','구독자수','조회수','비디오수'])
for column in df.columns[3:]:
    df[column] = df[column].apply(convert_unit)
df.head()

In [ ]:
driver.close()

In [ ]:
df.to_csv('YouTube랭킹.csv', index=False)